References:
https://github.com/bmabey/pyLDAvis
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

pyLDAvis is designed to help users interpret the topics in a topic model that has been fit to a corpus of text data. The
package extracts information from a fitted LDA topic model to inform an interactive web-based visualization.
The visualization is intended to be used within an IPython notebook but can also be saved to a stand-alone HTML file
for easy sharing

Reference: https://buildmedia.readthedocs.org/media/pdf/pyldavis/latest/pyldavis.pdf

Python libraries needed:
- pandas
- nltk
corpus to be download using nltk.download()
- stopwords
- wordnet
- gensim
- pyldavis

Import Ashville reviews dataset
About the dataset
The reviews dataset contains information about the reviews given by reviewers as of April 2018

In [2]:
#Import the dataset
import pandas as pd
listings = pd.read_csv("listings1.csv")
listings.head()
import spacy

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm")

In [3]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,38585,https://www.airbnb.com/rooms/38585,20180418164707,2018-04-18,Charming Victorian twin BR for 2,Per the City Council of Asheville. Due to the ...,"Charming room with 2 twin size beds, furnished...",Per the City Council of Asheville. Due to the ...,none,Our North Asheville neighborhood stretches alo...,...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",f,f,moderate,t,t,1,1.20
1,67870,https://www.airbnb.com/rooms/67870,20180418164707,2018-04-18,Your Own Floor! 1 bdrm W. Asheville,Your own floor and entrance! Lots of room with...,"Finished basement with living room, bedroom, ...",Your own floor and entrance! Lots of room with...,none,Our neighborhood is a quiet one on the west si...,...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",t,f,moderate,f,f,2,2.16
2,80713,https://www.airbnb.com/rooms/80713,20180418164707,2018-04-18,"[DOWNTOWN] Eco~Goddess Sanctuary, Monthly Rentals","Our beautiful, natural Sanctuary is a 1.3 acre...","FEATURED in WNC Magazine, Mountain Xpress, The...","Our beautiful, natural Sanctuary is a 1.3 acre...",none,I've lived in this wonderfully diverse & safe ...,...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",f,f,strict,f,f,2,0.49
3,80905,https://www.airbnb.com/rooms/80905,20180418164707,2018-04-18,French Industrial Chic Loft,Let yourself melt into the delectable décor of...,Have you ever gazed at dreamy photos in a maga...,Let yourself melt into the delectable décor of...,none,"Quiet, tucked away from the hustle and bustle ...",...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",f,f,super_strict_60,f,f,13,0.96
4,86980,https://www.airbnb.com/rooms/86980,20180418164707,2018-04-18,[Downtown] Goddess Sanctuary & Eco-Urban Homes...,"Our beautiful, natural guesthouse is part of a...","FEATURED in WNC Magazine, Mountain Xpress, The...","Our beautiful, natural guesthouse is part of a...",none,I've lived in this wonderfully diverse & safe ...,...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",f,f,strict,f,f,2,0.55


##Business Problem

Perform topic modelling on reviews of Ashville Airbnb data

We need to remove stopwords and perform lemmitization on our reviews, we need to import required libraries

In [4]:
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kanum\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
import re
import numpy as np
import pandas as pd
from pprint import pprint


In [6]:

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


In [7]:

# spacy for lemmatization
import spacy

In [8]:


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline


In [9]:

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [10]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use']) # We can add the extra words we want to remove from our dataset here

In [11]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,38585,https://www.airbnb.com/rooms/38585,20180418164707,2018-04-18,Charming Victorian twin BR for 2,Per the City Council of Asheville. Due to the ...,"Charming room with 2 twin size beds, furnished...",Per the City Council of Asheville. Due to the ...,none,Our North Asheville neighborhood stretches alo...,...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",f,f,moderate,t,t,1,1.20
1,67870,https://www.airbnb.com/rooms/67870,20180418164707,2018-04-18,Your Own Floor! 1 bdrm W. Asheville,Your own floor and entrance! Lots of room with...,"Finished basement with living room, bedroom, ...",Your own floor and entrance! Lots of room with...,none,Our neighborhood is a quiet one on the west si...,...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",t,f,moderate,f,f,2,2.16
2,80713,https://www.airbnb.com/rooms/80713,20180418164707,2018-04-18,"[DOWNTOWN] Eco~Goddess Sanctuary, Monthly Rentals","Our beautiful, natural Sanctuary is a 1.3 acre...","FEATURED in WNC Magazine, Mountain Xpress, The...","Our beautiful, natural Sanctuary is a 1.3 acre...",none,I've lived in this wonderfully diverse & safe ...,...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",f,f,strict,f,f,2,0.49
3,80905,https://www.airbnb.com/rooms/80905,20180418164707,2018-04-18,French Industrial Chic Loft,Let yourself melt into the delectable décor of...,Have you ever gazed at dreamy photos in a maga...,Let yourself melt into the delectable décor of...,none,"Quiet, tucked away from the hustle and bustle ...",...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",f,f,super_strict_60,f,f,13,0.96
4,86980,https://www.airbnb.com/rooms/86980,20180418164707,2018-04-18,[Downtown] Goddess Sanctuary & Eco-Urban Homes...,"Our beautiful, natural guesthouse is part of a...","FEATURED in WNC Magazine, Mountain Xpress, The...","Our beautiful, natural guesthouse is part of a...",none,I've lived in this wonderfully diverse & safe ...,...,f,NaN,"{""NORTH CAROLINA"","" NORTH CAROLINA"","" BUNCOMBE""}",f,f,strict,f,f,2,0.55


In [13]:
# Convert to list
data = listings.description.values.tolist()
pprint(data[:1])

['Per the City Council of Asheville. Due to the definite ban of short-term '
 'vacation rentals and accessory dwelling units in residentially zoned areas '
 'since January 9, 2018 in Asheville, which applies to downtown and several '
 'areas outside the city center, and carries hefty fines, I wish to assure my '
 'prospective guests that my "homestay" is totally legal as this is my primary '
 'residence, which I share with you, and I am not located in any of the '
 'aforementioned zoning areas. Charming room with 2 twin size beds, furnished '
 'with beautiful Victorian antiques on the 2nd floor with view of a lush '
 'forest and seasonally the Blue Ridge Mountains ... and the sunrise if you '
 'are an early bird ! There is a closet with hangers, a dresser and a '
 'nightstand to store your clothes and belongings. Your own private bathroom '
 'with tub/shower combination is adjacent to your room . We have central air '
 'conditioning and heating (HVAC). The house exudes old fashioned ch

In [14]:
#Tokenize words and Clean-up text
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['per', 'the', 'city', 'council', 'of', 'asheville', 'due', 'to', 'the', 'definite', 'ban', 'of', 'short', 'term', 'vacation', 'rentals', 'and', 'accessory', 'dwelling', 'units', 'in', 'residentially', 'zoned', 'areas', 'since', 'january', 'in', 'asheville', 'which', 'applies', 'to', 'downtown', 'and', 'several', 'areas', 'outside', 'the', 'city', 'center', 'and', 'carries', 'hefty', 'fines', 'wish', 'to', 'assure', 'my', 'prospective', 'guests', 'that', 'my', 'homestay', 'is', 'totally', 'legal', 'as', 'this', 'is', 'my', 'primary', 'residence', 'which', 'share', 'with', 'you', 'and', 'am', 'not', 'located', 'in', 'any', 'of', 'the', 'aforementioned', 'zoning', 'areas', 'charming', 'room', 'with', 'twin', 'size', 'beds', 'furnished', 'with', 'beautiful', 'victorian', 'antiques', 'on', 'the', 'nd', 'floor', 'with', 'view', 'of', 'lush', 'forest', 'and', 'seasonally', 'the', 'blue', 'ridge', 'mountains', 'and', 'the', 'sunrise', 'if', 'you', 'are', 'an', 'early', 'bird', 'there', 'is',

In [15]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['per', 'the', 'city', 'council', 'of', 'asheville', 'due', 'to', 'the', 'definite', 'ban', 'of', 'short_term', 'vacation_rentals', 'and', 'accessory', 'dwelling', 'units', 'in', 'residentially', 'zoned', 'areas', 'since', 'january', 'in', 'asheville', 'which', 'applies', 'to', 'downtown', 'and', 'several', 'areas', 'outside', 'the', 'city', 'center', 'and', 'carries', 'hefty', 'fines', 'wish', 'to', 'assure', 'my', 'prospective', 'guests', 'that', 'my', 'homestay', 'is', 'totally', 'legal', 'as', 'this', 'is', 'my', 'primary_residence', 'which', 'share', 'with', 'you', 'and', 'am', 'not', 'located', 'in', 'any', 'of', 'the', 'aforementioned', 'zoning', 'areas', 'charming', 'room', 'with', 'twin', 'size', 'beds', 'furnished', 'with', 'beautiful', 'victorian', 'antiques', 'on', 'the', 'nd', 'floor', 'with', 'view', 'of', 'lush', 'forest', 'and', 'seasonally', 'the', 'blue_ridge', 'mountains', 'and', 'the', 'sunrise', 'if', 'you', 'are', 'an', 'early', 'bird', 'there', 'is', 'closet', 'w

In [16]:
#Remove Stopwords, Make Bigrams and Lemmatize
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['asheville', 'due', 'vacation', 'rental', 'accessory', 'dwell', 'unit', 'residentially', 'zone', 'area', 'apply', 'downtown', 'several', 'area', 'city', 'center', 'carry', 'hefty', 'fine', 'wish', 'assure', 'prospective', 'guest', 'homestay', 'totally', 'legal', 'primary_residence', 'share', 'locate', 'aforementioned', 'zoning', 'area', 'charming', 'room', 'size', 'bed', 'furnish', 'beautiful', 'victorian', 'antique', 'floor', 'view', 'forest', 'seasonally', 'blue_ridge', 'mountain', 'sunrise', 'early', 'bird', 'closet', 'hanger', 'dresser', 'store', 'clothe', 'belonging', 'private', 'bathroom', 'shower', 'combination', 'adjacent', 'room', 'central', 'heating', 'exude', 'old', 'fashioned', 'charm', 'warmth', 'coziness', 'even_though', 'new']]


In [19]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 3), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)]]


In [20]:
id2word[0]

'accessory'

In [21]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('accessory', 1),
  ('adjacent', 1),
  ('aforementioned', 1),
  ('antique', 1),
  ('apply', 1),
  ('area', 3),
  ('asheville', 1),
  ('assure', 1),
  ('bathroom', 1),
  ('beautiful', 1),
  ('bed', 1),
  ('belonging', 1),
  ('bird', 1),
  ('blue_ridge', 1),
  ('carry', 1),
  ('center', 1),
  ('central', 1),
  ('charm', 1),
  ('charming', 1),
  ('city', 1),
  ('closet', 1),
  ('clothe', 1),
  ('combination', 1),
  ('coziness', 1),
  ('downtown', 1),
  ('dresser', 1),
  ('due', 1),
  ('dwell', 1),
  ('early', 1),
  ('even_though', 1),
  ('exude', 1),
  ('fashioned', 1),
  ('fine', 1),
  ('floor', 1),
  ('forest', 1),
  ('furnish', 1),
  ('guest', 1),
  ('hanger', 1),
  ('heating', 1),
  ('hefty', 1),
  ('homestay', 1),
  ('legal', 1),
  ('locate', 1),
  ('mountain', 1),
  ('new', 1),
  ('old', 1),
  ('primary_residence', 1),
  ('private', 1),
  ('prospective', 1),
  ('rental', 1),
  ('residentially', 1),
  ('room', 2),
  ('seasonally', 1),
  ('several', 1),
  ('share', 1),
  ('shower', 

In [22]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=3, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [23]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.034*"downtown" + 0.022*"minute" + 0.014*"walk" + 0.014*"restaurant" + '
  '0.014*"neighborhood" + 0.012*"home" + 0.011*"mile" + 0.011*"asheville" + '
  '0.009*"locate" + 0.009*"close"'),
 (1,
  '0.027*"private" + 0.026*"room" + 0.024*"bedroom" + 0.014*"guest" + '
  '0.013*"home" + 0.013*"full" + 0.012*"bed" + 0.012*"bathroom" + '
  '0.012*"kitchen" + 0.011*"space"'),
 (2,
  '0.020*"home" + 0.016*"mountain" + 0.010*"modern" + 0.010*"make" + '
  '0.010*"experience" + 0.009*"view" + 0.008*"feel" + 0.007*"property" + '
  '0.007*"luxurious" + 0.006*"tiny"')]


In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\kanum\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.116899  0.156657       1        1  51.172958
0     -0.130658 -0.150958       2        1  41.544579
2      0.247558 -0.005699       3        1   7.282461, topic_info=     Category         Freq          Term        Total  loglift  logprob
term                                                                   
51    Default  1457.000000          room  1457.000000  30.0000  30.0000
81    Default  1398.000000          home  1398.000000  29.0000  29.0000
24    Default  1809.000000      downtown  1809.000000  28.0000  28.0000
68    Default  1329.000000       bedroom  1329.000000  27.0000  27.0000
43    Default   506.000000      mountain   506.000000  26.0000  26.0000
99    Default   632.000000    restaurant   632.000000  25.0000  25.0000
47    Default  1670.000000       private  1670.000000  24.0000  24.0000
221   Default   731.000000          full   731.000000  23.0000  23.0000
10    Default   669.000000           bed   669.000000  22.0000  22.0000
289   Default   673.000000  neighborhood   673.000000  21.0000  21.0000
92    Default  1229.000000        minute  1229.000000  20.0000  20.0000
149   Default   686.000000          walk   686.000000  19.0000  19.0000
317   Default   394.000000          make   394.000000  18.0000  18.0000
63    Default   384.000000          view   384.000000  17.0000  17.0000
685   Default   239.000000        modern   239.000000  16.0000  16.0000
453   Default   578.000000         large   578.000000  15.0000  15.0000
477   Default   334.000000          feel   334.000000  14.0000  14.0000
375   Default   470.000000          bath   470.000000  13.0000  13.0000
200   Default   405.000000         place   405.000000  12.0000  12.0000
132   Default   349.000000         local   349.000000  11.0000  11.0000
6     Default   549.000000     asheville   549.000000  10.0000  10.0000
274   Default   125.000000    experience   125.000000   9.0000   9.0000
166   Default   670.000000       kitchen   670.000000   8.0000   8.0000
584   Default   226.000000      property   226.000000   7.0000   7.0000
262   Default   298.000000       brewery   298.000000   6.0000   6.0000
8     Default   719.000000      bathroom   719.000000   5.0000   5.0000
237   Default   192.000000       amenity   192.000000   4.0000   4.0000
493   Default   270.000000          shop   270.000000   3.0000   3.0000
191   Default   348.000000         drive   348.000000   2.0000   2.0000
816   Default   355.000000        living   355.000000   1.0000   1.0000
...       ...          ...           ...          ...      ...      ...
4133   Topic3    10.888890         wowed    11.554705   2.5604  -6.5595
5164   Topic3    22.267769  sole_purpose    23.643406   2.5598  -5.8441
1115   Topic3    12.827525   opportunity    13.617888   2.5599  -6.3956
437    Topic3    13.217314     attention    14.109258   2.5544  -6.3657
171    Topic3    35.480965          loft    48.688915   2.3032  -5.3782
274    Topic3    74.637321    experience   125.077118   2.1034  -4.6346
377    Topic3    35.480801         cabin    51.559574   2.2460  -5.3782
384    Topic3    22.106583          flat    31.163296   2.2763  -5.8514
685    Topic3    79.234695        modern   239.375229   1.5141  -4.5748
43     Topic3   125.874008      mountain   506.418274   1.2276  -4.1119
460    Topic3    32.823376       situate    58.773750   2.0371  -5.4561
1578   Topic3    30.785303      building    54.594391   2.0468  -5.5202
609    Topic3    43.956268       retreat   108.414337   1.7169  -5.1640
61     Topic3    38.136711      vacation    82.618790   1.8466  -5.3061
660    Topic3    44.942242       comfort   118.721451   1.6483  -5.1419
391    Topic3    32.180401        nature    61.115051   1.9783  -5.4759
81     Topic3   152.780609          home  1398.717407   0.4054  -3.9182
317    Topic3    76.779396          make   394.793915   0.9823  -4.6063
63     Topic3  